In [1]:
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import ipywidgets as widgets


In [2]:
def make_row(N, mu, sigma2):
    row = np.random.normal(size=int(N), loc=mu, scale=sigma2 ** .5)
    return row



def check_6(row, mu, sigma2, alpha, m):
    if m == 0:
        m = int(1 + np.floor(3.22 * np.log10(len(row))))
    min = np.min(row)
    max = np.max(row)
    h = (max - min) / m
    delta_sum = 0
    for i in range(m):
        n = np.sum((row >= min + h * i) & (row <= min + h * (i + 1)))
        p = scipy.stats.norm.cdf(min + h * (i + 1), mu, sigma2) - scipy.stats.norm.cdf(min + h * i, mu, sigma2)
        delta = (n - len(row) * p) ** 2 / (len(row) * p)
        delta_sum += delta

    tau = scipy.stats.poisson.ppf(1 - alpha / 2, m - 1)
    print("δ = {:.2f}".format(delta_sum))
    print("tau = {:.2f}".format(tau))
    return np.abs(delta_sum) <= tau


In [3]:
out = widgets.Output()
row_out = widgets.Output()
num_row = []

def generate_row(btn):
    global num_row
    with row_out:
        num_row = make_row(N.value, mu_1.value, sigma2_1.value)
        row_out.clear_output()
        print("\n".join(list(map(str, num_row))))


def on_click(btn):
    out.clear_output()
    with out:
        if len(num_row) < 1:
            print("Пожалуйста сначала сгенерируйте выборку")
            return
        
        print("Проверка по критерию Пирсона:")
        if check_6(num_row, mu_2.value, sigma2_2.value, alpha.value, m.value):
            print("Выполняется гипотеза H0")
        else:
            print("Выполняется гипотеза H1")
        print()
        

def mk_btn(description, action):
    btn = widgets.Button(description=description, layout=widgets.Layout(width="200px"))
    btn.on_click(action)
    return btn

In [4]:
dist_label_1 = widgets.Label(value="Параметры распределения из которого будет генерироваться выборка")
N = widgets.FloatText(description='N:', value=20)
mu_1 = widgets.FloatText(description='μ:', value=0)
sigma2_1 = widgets.FloatText(description='σ²:', value=1)

generate_button = mk_btn("Сгенерировать выборку", generate_row)

dist_label_2 = widgets.Label(value="Параметры распределения F1")
mu_2 = widgets.FloatText(description='μ:', value=0)
sigma2_2 = widgets.FloatText(description='σ²:', value=1)

check_label = widgets.Label(value="Данные для проверки")
alpha = widgets.BoundedFloatText(description='α:', value=0.05, min=0.001, max=1)
m = widgets.BoundedIntText(description='m(0 для подсчёта по формуле Стерджеса):', value=0, min=0, max=1000000)

check_button = mk_btn("Проверить гипотезы", on_click)

widgets.VBox((dist_label_1, mu_1, sigma2_1, N, generate_button, dist_label_2, mu_2, sigma2_2, check_label, alpha, m, check_button))


In [5]:
out

Output()

In [99]:
row_out

Output()